## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> For Seller to update:[Title_of_your_product](Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Medical Text Translation (EN-ES)

- **Model**: `jsl-medical-translation-en-es`
- **Model Description**: Medical text translation between English (EN) and Spanish (ES). The model supports a maximum input length of 1024 tokens.


In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [2]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

In [4]:
model_name = "jsl-medical-translation-en-es"

real_time_inference_instance_type = "ml.m4.2xlarge"
batch_transform_inference_instance_type = "ml.m4.2xlarge"

## Using the Model Package

This model package can be used to translate text from English to Spanish or vice versa. To specify the translation direction, set the `TRANSLATION_DIRECTION` environment variable. The possible values for `TRANSLATION_DIRECTION` are:

- `'en_to_es'` for translating English to Spanish
- `'es_to_en'` for translating Spanish to English

If `TRANSLATION_DIRECTION` is not provided, the default value `'en_to_es'` will be used.


## 2. create a deployable model from the model package (EN ==> ES).

In [5]:
# Define Model and Endpoint configuration parameter

config = {
    'TRANSLATION_DIRECTION': 'en_to_es',
}


model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn, 
    sagemaker_session=sagemaker_session, 
    env=config
)

### 2.1 Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

#### A. Deploy the SageMaker model to an endpoint

In [6]:
# Deploy the model
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
)

----------!

Once endpoint has been created, you would be able to perform real-time inference.

In [7]:
import json
import pandas as pd
import os
import boto3

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

def process_data_and_invoke_realtime_endpoint(data, content_type, accept, translation_direction):

    content_type_to_format = {'application/json': 'json', 'application/jsonlines': 'jsonl'}
    input_format = content_type_to_format.get(content_type)
    if content_type not in content_type_to_format.keys() or accept not in content_type_to_format.keys():
        raise ValueError("Invalid content_type or accept. It should be either 'application/json' or 'application/jsonlines'.")

    i = 1
    input_dir = f'inputs/{translation_direction}/real-time/{input_format}'
    output_dir = f'outputs/{translation_direction}/real-time/{input_format}'
    s3_input_dir = f"{model_name}/validation-input/{translation_direction}/real-time/{input_format}"
    s3_output_dir = f"{model_name}/validation-output/{translation_direction}/real-time/{input_format}"

    input_file_name = f'{input_dir}/input{i}.{input_format}'
    output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    while os.path.exists(input_file_name) or os.path.exists(output_file_name):
        i += 1
        input_file_name = f'{input_dir}/input{i}.{input_format}'
        output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'


    os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
    os.makedirs(os.path.dirname(output_file_name), exist_ok=True)
    
    input_data = json.dumps(data, indent=4, ensure_ascii=False) if content_type == 'application/json' else data


    if content_type == 'application/json':
        with open(input_file_name, "w", encoding='utf-8') as f:
            f.write(input_data)
    else:
        with open(input_file_name, 'w', encoding='utf-8') as file:
            for line in input_data.splitlines():
                json_object = json.loads(line)
                file.write(json.dumps(json_object, ensure_ascii=False) + '\n')

    with open(input_file_name, 'rb') as data:
        s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_input_dir}/{os.path.basename(input_file_name)}", Body=data)

    # Invoke the SageMaker endpoint
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=input_data,
    )

    # Read response data
    response_data = json.loads(response["Body"].read().decode("utf-8")) if accept == 'application/json' else response['Body'].read().decode('utf-8')

    # Save response data to file
    with open(output_file_name, 'w') as f_out:
        if accept == 'application/json':
            json.dump(response_data, f_out, indent=4, ensure_ascii=False)
        else:
            for item in response_data.split('\n'):
                f_out.write(item + '\n')

    with open(output_file_name, 'rb') as data:
        s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_output_dir}/{os.path.basename(output_file_name)}", Body=data)

    return response_data

#### Initial Setup

In [8]:
english_text = "Primary empty sella turcica associated with diabetes insipidus and campimetric defect"
english_records = [
    "The patient was diagnosed with acute myocardial infarction.",
    "Chronic obstructive pulmonary disease is characterized by long-term breathing problems.",
]

### JSON


1. **Input format**: Single Text Document

Provide a single text document as a string.

  
  
```json
{
    "text": "Single text document"
}
```

2. **Input format**: Array of Text Documents

Use an array containing multiple text documents. Each element represents a separate text document.

```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ]
}
```

##### Example 1

In [9]:
input_json_data = {"text": english_text}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json", translation_direction="en-to-es")

pd.DataFrame(data)

,predictions
0,Sella turca vacía primaria asociada a diabetes insípida y defecto campimétrico.


##### Example 2

In [10]:
input_json_data = {"text": english_records} #The timeout for InvokeEndpoint is 60 seconds, so please carefully select the number of input texts you are sending in one request.

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json", translation_direction="en-to-es")

pd.DataFrame(data)

,predictions
0,El paciente fue diagnosticado de infarto agudo de miocardio.
1,La enfermedad pulmonar obstructiva crónica se caracteriza por problemas respiratorios a largo plazo.


#### JSON Lines

**Input format**: Provide input in JSON Lines format, where each line is a JSON object representing a text document along with any optional parameters.

```json
{"text": "Text document 1"}
{"text": "Text document 2"}
```

In [11]:
import json

def create_jsonl(records):
    json_records = []

    for text in records:
        record = {
            "text": text
        }
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record) for record in json_records)

    return json_lines

##### Example 1

In [12]:
input_jsonl_data = create_jsonl(english_records) #The timeout for InvokeEndpoint is 60 seconds, so please carefully select the number of input texts you are sending in one request.

data =  process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines", translation_direction="en-to-es")

print(data)

{"predictions": "El paciente fue diagnosticado de infarto agudo de miocardio."}
{"predictions": "La enfermedad pulmonar obstructiva crónica se caracteriza por problemas respiratorios a largo plazo."}


#### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [13]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

### 2.2 Batch inference

In [14]:
import json
import os


input_dir = 'inputs/en-to-es/batch'
json_input_dir = f"{input_dir}/json"
jsonl_input_dir = f"{input_dir}/jsonl"

output_dir = 'outputs/en-to-es/batch'
json_output_dir = f"{output_dir}/json"
jsonl_output_dir = f"{output_dir}/jsonl"

# JSON file names
validation_json_file_name = "input.json"

# JSONL file names
validation_jsonl_file_name = "input.jsonl"


os.makedirs(json_input_dir, exist_ok=True)
os.makedirs(jsonl_input_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)
os.makedirs(jsonl_output_dir, exist_ok=True)


validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/en-to-es/batch/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/en-to-es/batch/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/en-to-es/batch/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/en-to-es/batch/jsonl/"


def write_and_upload_to_s3(input_data, file_name, translation_direction):

    file_format = os.path.splitext(file_name)[1].lower()
    s3_key = f"{model_name}/validation-input/{translation_direction}/batch/{file_format[1:]}/{os.path.basename(file_name)}"

    if file_format == ".json":
        input_data_json = json.dumps(input_data, indent=4, ensure_ascii=False)

        with open(file_name, "w", encoding='utf-8') as f:
            f.write(input_data_json)
    elif file_format == ".jsonl":
        with open(file_name, 'w', encoding='utf-8') as file:
            for line in input_data.splitlines():
                json_object = json.loads(line)
                file.write(json.dumps(json_object, ensure_ascii=False) + '\n')

    with open(file_name, 'rb') as data:
        s3_client.put_object(Bucket=s3_bucket, Key=s3_key, Body=data)



input_jsonl_data = create_jsonl(english_records)
input_json_data ={"text": english_records}


write_and_upload_to_s3(input_jsonl_data, f"{jsonl_input_dir}/{validation_jsonl_file_name}",translation_direction = "en-to-es")
write_and_upload_to_s3(input_json_data, f"{json_input_dir}/{validation_json_file_name}", translation_direction = "en-to-es")

#### JSON

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [16]:
from urllib.parse import urlparse

def process_s3_json_output_and_save(validation_file_name):
    
    output_file_path = f"{json_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    df = pd.DataFrame(data)
    display(df)

    with open(output_file_path, 'w', encoding='utf-8') as f_out:
        json.dump(data, f_out, indent=4, ensure_ascii=False)

In [17]:
process_s3_json_output_and_save(validation_json_file_name)

,predictions
0,El paciente fue diagnosticado de infarto agudo de miocardio.
1,La enfermedad pulmonar obstructiva crónica se caracteriza por problemas respiratorios a largo plazo.


#### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [19]:
from urllib.parse import urlparse

def process_s3_jsonlines_output_and_save(validation_file_name):

    output_file_path = f"{jsonl_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        for item in data.split('\n'):
            f_out.write(item + '\n')

In [20]:
process_s3_jsonlines_output_and_save(validation_jsonl_file_name)

{"predictions": "El paciente fue diagnosticado de infarto agudo de miocardio."}
{"predictions": "La enfermedad pulmonar obstructiva crónica se caracteriza por problemas respiratorios a largo plazo."}


In [21]:
model.delete_model()

INFO:sagemaker:Deleting model with name: jsl-medical-translation-en-es-2024-07-03-14-25-17-522


## 3. create a deployable model from the model package (ES ==> EN).

In [22]:
# Define Model and Endpoint configuration parameter

config = {
    'TRANSLATION_DIRECTION': 'es_to_en',
}


model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn, 
    sagemaker_session=sagemaker_session, 
    env=config
)

### 3.1 Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

#### A. Deploy the SageMaker model to an endpoint

In [ ]:
# Deploy the model
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
)

Once endpoint has been created, you would be able to perform real-time inference.

#### Initial Setup

In [24]:
spanish_text = "Evaluación clínico-radiológica de la técnica de aloinjerto compactado y vástago cementado en cirugías de revisión de cadera."
spanish_records = [
    "El paciente fue sometido a una tomografía computarizada del abdomen.",
    "La hipertensión arterial es una condición crónica que puede causar daño a varios órganos."
]

#### JSON

##### Example 1

In [25]:
input_json_data = {"text": spanish_text}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json", translation_direction="es-to-en")

pd.DataFrame(data)

,predictions
0,Clinico-radiological evaluation of the compact allo-graft and cemented stent technic in hip revision surgeries.


##### Example 2

In [26]:
input_json_data = {"text": spanish_records} #The timeout for InvokeEndpoint is 60 seconds, so please carefully select the number of input texts you are sending in one request.

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json", translation_direction="es-to-en")

pd.DataFrame(data)

,predictions
0,The patient was subjected to computed tomography of the abdomen.
1,Arterial hypertension is a chronic condition that can cause damage to various organs.


#### JSON Lines

##### Example 1

In [27]:
input_jsonl_data = create_jsonl(spanish_records) #The timeout for InvokeEndpoint is 60 seconds, so please carefully select the number of input texts you are sending in one request.

data =  process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines", translation_direction="es-to-en")

print(data)

{"predictions": "The patient was subjected to computed tomography of the abdomen."}
{"predictions": "Arterial hypertension is a chronic condition that can cause damage to various organs."}


#### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [28]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

INFO:sagemaker:Deleting endpoint with name: jsl-medical-translation-en-es
INFO:sagemaker:Deleting endpoint configuration with name: jsl-medical-translation-en-es


## 3.2 Batch inference

In [29]:
import json
import os


input_dir = 'inputs/es-to-en/batch'
json_input_dir = f"{input_dir}/json"
jsonl_input_dir = f"{input_dir}/jsonl"

output_dir = 'outputs/es-to-en/batch'
json_output_dir = f"{output_dir}/json"
jsonl_output_dir = f"{output_dir}/jsonl"

# JSON file names
validation_json_file_name = "input.json"

# JSONL file names
validation_jsonl_file_name = "input.jsonl"


os.makedirs(json_input_dir, exist_ok=True)
os.makedirs(jsonl_input_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)
os.makedirs(jsonl_output_dir, exist_ok=True)


validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/es-to-en/batch/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/es-to-en/batch/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/es-to-en/batch/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/es-to-en/batch/jsonl/"



input_jsonl_data = create_jsonl(spanish_records)
input_json_data ={"text": spanish_records}


write_and_upload_to_s3(input_jsonl_data, f"{jsonl_input_dir}/{validation_jsonl_file_name}",translation_direction = "es-to-en")
write_and_upload_to_s3(input_json_data, f"{json_input_dir}/{validation_json_file_name}", translation_direction = "es-to-en")

#### JSON

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [31]:
process_s3_json_output_and_save(validation_json_file_name)

,predictions
0,The patient was subjected to computed tomography of the abdomen.
1,Arterial hypertension is a chronic condition that can cause damage to various organs.


#### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [35]:
process_s3_jsonlines_output_and_save(validation_jsonl_file_name)

{"predictions": "The patient was subjected to computed tomography of the abdomen."}
{"predictions": "Arterial hypertension is a chronic condition that can cause damage to various organs."}


In [ ]:
model.delete_model()

### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

